In [14]:
from __future__ import print_function
import numpy as np
import numpy as np
import random
from sklearn import preprocessing
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, RobustScaler, StandardScaler
import sys
sys.setrecursionlimit(1500)

In [68]:
def train_scaler(X):
    scaler = MaxAbsScaler()
    scaler.fit(X)
    return scaler    

In [69]:
def load_training_data(csv_file_name, undersample=False):
    """Loads a CSV file produced by the Java feature generator."""
    X_raw = []
    y_raw = []
    indices = (1,2,3,4,5,6,7,8,9,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29)
    random.seed(21)
    with open(csv_file_name, 'r') as f:
        for line_number, line in enumerate(f.readlines()):
            parts = line[:-1].split(",")
            if undersample:
                if int(parts[0])==0:
                    if random.random() < 0.152: # no undersampling for now
                        y_raw.append(int(parts[0]))
                        features_np = np.array([float(f) for f in parts[1:]])
                        X_raw.append(features_np) 
                else:
                    y_raw.append(int(parts[0]))
                    features_np = np.array([float(f) for f in parts[1:]])
                    X_raw.append(features_np)  
            else:
                y_raw.append(int(parts[0]))
                features_np = np.array([float(f) for f in parts[1:]])
                X_raw.append(features_np)                    
        X_raw = np.array(X_raw)
        y_raw = np.array(y_raw)
        
        X_raw = X_raw[:,indices]

        print("Feature shape: {0}".format(X_raw.shape))
        print("Label shape: {0}".format(y_raw.shape))
        return np.nan_to_num(X_raw), np.nan_to_num(y_raw)

In [70]:
csv_file_name_train = "../../../../data/nonempty_train_dict_strict.csv"
csv_file_name_test = "../../../../data/nonempty_lululu_dict_strict.csv"

In [71]:
X_train, y_train = load_training_data(csv_file_name_train,True)
X_test, y_test = load_training_data(csv_file_name_test)

Feature shape: (76, 27)
Label shape: (76,)
Feature shape: (246, 27)
Label shape: (246,)


In [72]:
scaler = train_scaler(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
train_pos = np.sum(y_train == 1)
test_pos = np.sum(y_test == 1)
print("We have {0} positive labels in training set.".format(train_pos))
print("We have {0} positive labels in validation set.".format(test_pos))

We have 34 positive labels in training set.
We have 26 positive labels in validation set.


In [43]:
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR



In [57]:
#clf = SGDClassifier(alpha=0.002, loss='log', penalty='l2', n_iter=1000)
#clf = LinearDiscriminantAnalysis()
clf = SVC(kernel='rbf', C=0.75, probability=True, random_state=21)
clf.fit(X_train,y_train)
th = 0.9

y_prob = clf.predict_proba(X_test)
y_predict = np.zeros(len(y_test))
for i in range(len(y_test)):
    if y_prob[i,1] >= th:
        y_predict[i] = 1
        
fp = fn = tp = 0 
for i in range(len(y_predict)):
    if y_predict[i] == y_test[i] == 1:
        tp += 1
        #print(y_prob[i])
    if y_predict[i] == 1 and y_test[i] == 0:
        fp += 1
        #print(i)
        #print(y_prob[i])
    if y_predict[i] == 0 and y_test[i] == 1:
        fn += 1
        #print(i)
        #print("+++++++++++++++++++++++++++++++{0}".format(y_prob[i]))
print("True empty predictions {0}!".format(tp))
print("{0} were predicted to be empty but were not; {1} predicted to be nonempty but were not;".format(fp,fn))

True empty predictions 7!
6 were predicted to be empty but were not; 19 predicted to be nonempty but were not;


In [59]:
def eval(filename):
    X_test, y_test = load_training_data(filename)    
    fp = fn = tp = 0 
    for i in range(len(y_test)):
        if X_test[i,1] == 0:
            predict = 1
        else: 
            predict = 0
        if predict == y_test[i] == 1:
            tp += 1
            #print(y_prob[i])
        if predict == 1 and y_test[i] == 0:
            fp += 1
            #print(i)
            #print(y_prob[i])
        if predict == 0 and y_test[i] == 1:
            fn += 1
            #print(i)
    print("True empty predictions {0}!".format(tp))
    print("{0} were predicted to be empty but were not; {1} predicted to be nonempty but were not;".format(fp,fn))

In [60]:
eval("../../../../data/nonempty_train_dict_strict.csv")

Feature shape: (322, 27)
Label shape: (322,)
True empty predictions 2!
3 were predicted to be empty but were not; 32 predicted to be nonempty but were not;


In [61]:
eval("../../../../data/nonempty_valid_dict.csv")

Feature shape: (249, 27)
Label shape: (249,)
True empty predictions 3!
6 were predicted to be empty but were not; 25 predicted to be nonempty but were not;


In [64]:
eval("../../../../data/nonempty_lululu_dict.csv")

Feature shape: (246, 27)
Label shape: (246,)
True empty predictions 3!
6 were predicted to be empty but were not; 23 predicted to be nonempty but were not;
